# To Track the chatbot

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() # for loading all keys

os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']= os.getenv('langsmith_api')
os.environ['LANGCHAIN_PROJECT']="RAG ChatBot"

# Initialize LLM

In [2]:
from langchain_groq import ChatGroq


os.environ["GROQ_API_KEY"] = os.getenv('groq_api')


llm = ChatGroq(model="llama3-8b-8192")

llm.invoke("hi")

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.020833333, 'prompt_time': 0.001640051, 'queue_time': 0.013060158, 'total_time': 0.022473384}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-d024b80d-3f0b-450c-93cc-f6583d6e3896-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

# load saved vectore store

In [3]:
import pickle

# Load vectorstore from the pickle file
with open("E:\\align_minds_assessement\\test_pdf_&_retriever\\vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)


In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 7})
retriever.invoke("which database to use?")

[Document(metadata={'source': 'E:\\align_minds_assessement\\test_pdf_&_retriever\\Alignminds-AI-Machine-Test.pdf', 'page': 0}, page_content='of retrieval-based search with response generation. • Vector Database: FAISS for effective vector-based data storage and retrieval. • LLM API Integration: Use a suitable LLM API, including options like Gemini, OpenAI, or Hugging Face models. • Document Loading: Utilize document loaders for processing data sources (PDFs, Word documents). • API Design: Build a user-interactive API endpoint for seamless querying. • LangChain Usage: Leverage LangChain’s create_retrieval_chain to handle document'),
 Document(metadata={'source': 'E:\\align_minds_assessement\\test_pdf_&_retriever\\Alignminds-AI-Machine-Test.pdf', 'page': 1}, page_content='the main tasks within 3 days. Share your progress by the deadline. Important Notes • Data Source Flexibility: Use any publicly available documents for data retrieval. • API Key Security: Use your own API key, but do not

# retrieval chain creation

In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "can you tell about the assessment?"})
print(response["answer"])

The assessment is for a Retrieval-Augmented Generation (RAG) chatbot that uses FAISS as a vector database and an LLM API (such as Gemini, OpenAI's GPT, or Hugging Face models) with LangChain's create_retrieval_chain. The goal is to create a chatbot that can provide accurate and contextually relevant responses to user queries.


# Add Memory

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



In [8]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "can you tell about the assessment?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "how many days to complete?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])



According to the provided context, the main tasks within the 3-day deadline.


# GUI for the chatbot

In [ ]:
import gradio as gr
from langchain_core.messages import AIMessage, HumanMessage

# Initialize chat history
chat_history = []

# Define the RAG chain function with error handling
def chat_with_rag(question):
    global chat_history
    try:
        # Invoke the RAG chain with the user's question and chat history
        response = rag_chain.invoke({"input": question, "chat_history": chat_history})

        # Add the question and response to chat history
        chat_history.extend(
            [
                HumanMessage(content=question),
                AIMessage(content=response["answer"]),
            ]
        )

        # Return the AI's answer
        return response["answer"]

    except Exception as e:
        # Handle errors gracefully and provide feedback to the user
        return f"An error occurred: {str(e)}"

# Define Gradio chatbot function
def gradio_chatbot(user_input):
    # Get the response from RAG model
    response = chat_with_rag(user_input)
    return response

# Set up the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("<h3>Chatbot with RAG Chain</h3>")
    
    # Chat history and user input
    chatbot = gr.Chatbot()
    with gr.Row():
        user_input = gr.Textbox(show_label=False, placeholder="Ask me something...")
        submit_button = gr.Button("Send")

    # Action to take on submit
    def respond(message, chat_history):
        response = gradio_chatbot(message)
        chat_history.append((message, response))
        return "", chat_history

    # Trigger the chatbot response
    submit_button.click(respond, [user_input, chatbot], [user_input, chatbot])

# Launch the Gradio app
demo.launch(debug=True)
